In [1]:
# SCRAPING WIKIPEDIA

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
for row_cell in table.find_all('tr'):
    row_data=[]
    for data_cell in row_cell.find_all('td'):
        row_data.append(data_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
    
df=df[df['Borough']!='Not assigned']
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])


temp_df=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'}, inplace=True)
df_cleaned = pd.merge(df, temp_df, on='PostalCode')
df_cleaned.drop(['Neighborhood'],axis=1, inplace=True)
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.rename(columns={'Neighborhood_joined':'Neighborhood'}, inplace=True)

df_cleaned

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [3]:
df_cleaned.shape

(103, 3)

In [4]:
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 98 kB 9.7 MB/s  eta 0:00:01


In [5]:
import geocoder # import geocoder

def get_geo(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [6]:
geocoder_df=pd.read_csv('http://cocl.us/Geospatial_data')
geocoder_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_cleaned = pd.merge(geocoder_df, df_cleaned, on='PostalCode')
geo_final =geo_cleaned[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
